In [1]:
from transformers import BertTokenizer, BertForTokenClassification
import os, pickle, torch
from tqdm import tqdm

c:\Kyode\PersonalProjects\HateSpeechModelCombination\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained('GroNLP/hateBERT')
model = BertForTokenClassification.from_pretrained('GroNLP/hateBERT')

c:\Kyode\PersonalProjects\HateSpeechModelCombination\venv\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gener\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForTokenClassification: 

In [3]:
dynahate_dataset_path = "..\\Data_Preprocessing\\PreProcessed_Data\\DynaHate\\"
latenthatred_dataset_path = "..\\Data_Preprocessing\\PreProcessed_Data\\Latent_Hatred\\"
olid_dataset_path = "..\\Data_Preprocessing\\PreProcessed_Data\\OLID\\"

dynahate_embeddings_path = "Model_Embeddings\\HateBERT\\DynaHate\\"
latenthatred_embeddings_path = "Model_Embeddings\\HateBERT\\Latent_Hatred\\"
olid_embeddings_path = "Model_Embeddings\\HateBERT\\OLID\\"

In [4]:
def dump_embeddings(dataset = "dynahate", task = "train"):
    dataset_path = None
    curr_task = None
    embedding_path = None
    text_column = 0
    text = []
    labels = []
    embeddings = []
    
    if dataset == "dynahate":
        dataset_path = dynahate_dataset_path
        if task == "train":
            curr_task = "DynaHate_Training"
        elif task == "dev":
            curr_task = "DynaHate_Val"
        else:
            curr_task = "DynaHate_Test"
        embedding_path = os.path.join(dynahate_embeddings_path, curr_task)
    elif dataset == "latenthatred":
        dataset_path = latenthatred_dataset_path
        if task == "train":
            curr_task = "LatentHatred_Training"
        elif task == "dev":
            curr_task = "LatentHatred_Val"
        else:
            curr_task = "LatentHatred_Test"
        embedding_path = os.path.join(latenthatred_embeddings_path, curr_task)
    else:
        text_column = 1
        dataset_path = olid_dataset_path
        if task == "train":
            curr_task = "OLID_Training"
        elif task == "dev":
            curr_task = "OLID_Val"
        else:
            curr_task = "OLID_Test"
        embedding_path = os.path.join(olid_embeddings_path, curr_task)
    dataset_path = os.path.join(dataset_path, curr_task + ".txt")
    with open(dataset_path, "r", encoding="utf8") as file:
        temp = file.readlines()
    file.close()
    for each in temp[1:]:
        curr = each.split()
        text.append(' '.join(curr[text_column:-1]))
        labels.append(curr[-1])
    
    for each in tqdm(text):
        tokenized_text = tokenizer(each, return_tensors = "pt")
        inputs = {k: v for k, v in tokenized_text.items()}
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs[-1]
        cls_state = hidden_states[0][0, 0, :]
        embeddings.append(cls_state)
    
    embeddings = torch.stack(embeddings)
    print(embeddings.shape)

    with open(embedding_path, "wb+") as file:
        pickle.dump(embeddings, file)
    file.close()

In [5]:
for dataset in ["dynahate", "latenthatred", "olid"]:
    for task in ["train", "dev", "test"]:
        dump_embeddings(dataset = dataset, task = task)

100%|██████████| 32924/32924 [1:27:45<00:00,  6.25it/s]  


torch.Size([32924, 768])


100%|██████████| 4100/4100 [08:09<00:00,  8.38it/s]


torch.Size([4100, 768])


100%|██████████| 4120/4120 [08:09<00:00,  8.41it/s]


...

torch.Size([10592, 768])


100%|██████████| 2648/2648 [06:16<00:00,  7.03it/s]


torch.Size([2648, 768])


100%|██████████| 860/860 [02:24<00:00,  5.96it/s]


torch.Size([860, 768])
